<a href="https://colab.research.google.com/github/monya-9/deep-learning-practice/blob/main/17_bert_lora_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA 적용하여 BERT fine-tuning

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes evaluate -q

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch
import evaluate

In [ ]:
# --------------------------
# 1. 데이터 준비
# --------------------------
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 데이터 샘플 줄이기 (빠른 테스트용)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
# --------------------------
# 2. BERT 모델 불러오기
# --------------------------
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# --------------------------
# 3. LoRA 설정
# --------------------------
lora_config = LoraConfig(
    r=8,              # low-rank 차원
    lora_alpha=32,    # 스케일링
    target_modules=["query", "value"],  # 어댑터 적용 대상 모듈
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)

- r=8 : 저차원 적응 행렬 차원

- target_modules=["query", "value"] : Transformer의 Query/Value 가중치에만 적용

- lora_alpha : 학습률 스케일링

- 전체 BERT 가중치는 고정 → 메모리 효율 + 빠른 학습

In [ ]:
# --------------------------
# 4. 지표 정의
# --------------------------
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }

In [ ]:
# --------------------------
# 5. Trainer 정의
# --------------------------
training_args = TrainingArguments(
    output_dir="./results_lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=5e-4,
    logging_dir="./logs",
    report_to="none",  # W&B 비활성화
    fp16=True           # GPU mixed precision
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-83397586.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


- HuggingFace Trainer 그대로 사용 가능

- LoRA 모델도 동일 인터페이스로 학습 가능

In [ ]:
# --------------------------
# 6. 학습 + 평가
# --------------------------
trainer.train()
results = trainer.evaluate()
print("\n✅ 평가 결과:", results)

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.390441,0.820000,0.820023



✅ 평가 결과: {'eval_loss': 0.3904406428337097, 'eval_accuracy': 0.82, 'eval_f1': 0.8200230414746544, 'eval_runtime': 4.5483, 'eval_samples_per_second': 219.864, 'eval_steps_per_second': 27.483, 'epoch': 1.0}
